In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
from shapely.geometry import Point
def intpt_func(row):
    return Point(row['INTPTLON'], row['INTPTLAT'])

In [4]:
#loading LODES data

ham_lodes = pd.read_csv('../data/hamilton_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
ham_lodes.h_geocode = ham_lodes.h_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.astype(str)
ham_lodes.h_geocode = ham_lodes.h_geocode.astype(str)

#loading Hamilton county geodata
ham_cbg = pd.read_csv('../data/ham_cbg.csv')
ham_cbg['intpt'] = ham_cbg[['INTPTLAT', 'INTPTLON']].apply(lambda p: intpt_func(p), axis=1)
ham_cbg = gpd.GeoDataFrame(ham_cbg, geometry=gpd.GeoSeries.from_wkt(ham_cbg.geometry))
ham_cbg.GEOID = ham_cbg.GEOID.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
#loading residential buildings
res_build = pd.read_csv('../data/ham_residential_buildings2.csv', index_col=0)
res_build = gpd.GeoDataFrame(res_build, geometry=gpd.GeoSeries.from_wkt(res_build.geometry))
res_build['location'] = res_build.geometry.apply(lambda p: [p.y, p.x])

#loading work buildings
com_build = pd.read_csv('../data/work_loc_poi_com_civ.csv', index_col=0)
com_build = gpd.GeoDataFrame(com_build, geometry=gpd.GeoSeries.from_wkt(com_build.geometry))
com_build['location'] = com_build.geometry.apply(lambda p: [p.y, p.x])
com_build = com_build.reset_index()
com_build.GEOID = com_build.GEOID.astype(str)

#loading all buildings (MS dataset)
ms_build = pd.read_csv('../data/ham_buildings_MS.csv')
ms_build = gpd.GeoDataFrame(ms_build, geometry=gpd.GeoSeries.from_wkt(ms_build.geo_centers))
ms_build.GEOID = ms_build.GEOID.astype(str)
ms_build['location'] = ms_build.geometry.apply(lambda p: [p.y, p.x])

In [6]:
#aggregating total jobs for each combination of home and work cbg 
ham_lodes = ham_lodes.groupby(['h_geocode', 'w_geocode']).agg(total_jobs=('total_jobs', sum)).reset_index().merge(ham_cbg[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').rename({'geometry':'home_geom'}, axis=1).drop('GEOID', axis=1).merge(ham_cbg[['GEOID', 'geometry']], left_on='w_geocode', right_on='GEOID').rename({'geometry':'work_geom'}, axis=1).drop('GEOID', axis=1).sort_values('total_jobs', ascending=False).reset_index(drop=True)
ham_lodes = gpd.GeoDataFrame(ham_lodes)

In [9]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [98]:
#generating array of start and return times (in 15 min intervals)
from datetime  import datetime, timedelta
times_morning = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 7), datetime(2016, 9, 1, 9, 10), 
       timedelta(minutes=15))]
times_evening = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 16), datetime(2016, 9, 1, 18, 10), 
       timedelta(minutes=15))]

In [11]:
res_build.GEOID = res_build.GEOID.astype(str)
com_build.GEOID = com_build.GEOID.astype(str)

In [109]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

#setting the random seed
np.random.seed(42)
random.seed(42)

prob_matrix = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(ham_lodes.iterrows(), total=ham_lodes.shape[0]):

    res = res_build[res_build.GEOID == movement.h_geocode].reset_index(drop=True)
    if res.empty:
        res = ms_build[ms_build.GEOID == movement.h_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.w_geocode].reset_index(drop=True)
    if com.empty:
        com = ms_build[ms_build.GEOID == movement.w_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)
        
    r = res
    c = com
   
    for job in range(movement.total_jobs):
     
        if c.empty:
            c = com
        if r.empty:
            r = res

        rand_r = random.randrange(0, r.shape[0])
        rand_c = random.randrange(0, c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)
        
        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[job, 'h_geocode'] = movement.h_geocode
        temp.loc[job, 'w_geocode'] = movement.w_geocode
        temp.loc[job, 'total_jobs'] = movement.total_jobs
        temp.loc[job, 'home_loc_lat'] = r_df.location[0]
        temp.loc[job, 'home_loc_lon'] = r_df.location[1]
        temp.loc[job, 'work_loc_lat'] = c_df.location[0]
        temp.loc[job, 'work_loc_lon'] = c_df.location[1]
        temp.loc[job, 'go_time'] = time_slot1[0]
        temp.loc[job, 'go_time_str'] = time_slot1[0].strftime('%H:%M')
        temp.loc[job, 'return_time'] = time_slot2[0]
        temp.loc[job, 'return_time_str'] = time_slot2[0].strftime('%H:%M')

        prob_matrix = prob_matrix.append(temp, ignore_index=True)

  0%|          | 0/14282 [00:00<?, ?it/s]

In [110]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [111]:
# convert the lat and lon points to shapely Points
prob_matrix['home_geom'] = prob_matrix[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix['work_geom'] = prob_matrix[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)
prob_matrix.h_geocode = prob_matrix.h_geocode.astype(str)
prob_matrix.w_geocode = prob_matrix.w_geocode.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [115]:
# prob_matrix.to_csv('lodes_combinations.csv', index=False)
prob_matrix.to_parquet('lodes_combinations.parquet', index=False)